In [1]:
import numpy as np
from pyecharts import Line,Grid,Bar,TreeMap
import datetime
from pandas.tseries.offsets import Day
import pandas as pd

path = r"E:\FMM\2019\2-13\采购台账2019年1月份.xlsx"
# dataf= pd.read_excel(path,skiprows=1,sheetname = "采购台账2019年1月份（呼家楼和东四）")
dataf= pd.read_excel(path,skiprows=1)
df = dataf.iloc[:,[0,5,6,7,19,20,21,22]]
print(df.head())
print(df.info())
# 数据处理规则
# 新建列
# 1.类别：if新物料类别=‘蔬菜类’：类别=新物料类别  else:类别=原物料类别
# 2.单价:if单价=空 ：单价=金额/实际采购 else:单价=单价（含税/运/包） || if 单价！=数字：drop一行  
# df['类别'] = na
#添加类别列 
from  tqdm import *
df['类别'] = ''
for i in tqdm(range(len(df))):
    if df.iloc[i,1] == '蔬菜类':
        df['类别'][i]  = df.iloc[i,1]
    else:
        df['类别'][i] = df.iloc[i,2]
df.head()
df['单价'] = ''

#添加单价列
for i in tqdm(range(len(df))):
  
    if pd.isnull(df.iloc[i,5]):
        df['单价'][i]=df.iloc[i,6]/df.iloc[i,4]
    else:
        df['单价'][i] = df.iloc[i,5]
        
# 单价nan值删除
# df['单价'].isnull().value_counts()
df['单价']=df['单价'].fillna('999')
slist = df[(df.单价=='999')].index.tolist()
df = df.drop(slist)
df['单价'] = df['单价'].astype(np.float64)
df.dropna(axis = 0,how = 'any',inplace = True)#删除包含空值的行

# ！截止到如此数据预处理基本完成 ，下面针对不同的图，提取处理不同的数据

      采购\n日期 新物料类别 原物料\n类别 物料\n描述  实际\n采购  单价（含税/运/包）     金额  实采\n差异
0 2019-01-01   蔬菜类   叶菜类蔬菜   黄心白菜    50.0         1.6   80.0    0.00
1 2019-01-01   蔬菜类   根茎类蔬菜    白萝卜    48.0         1.4   67.0   -0.28
2 2019-01-01   蔬菜类   根茎类蔬菜    白洋葱     0.5         1.8    0.9   -0.13
3 2019-01-01   蔬菜类   根茎类蔬菜   青梗菜花    80.0         5.2  416.0    1.41
4 2019-01-01   蔬菜类    葱姜蒜类     大葱    23.0         3.0   69.0   -0.97
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2886 entries, 0 to 2885
Data columns (total 8 columns):
采购
日期         2886 non-null datetime64[ns]
新物料类别         2813 non-null object
原物料
类别        2813 non-null object
物料
描述         2813 non-null object
实际
采购         2241 non-null float64
单价（含税/运/包）    2055 non-null float64
金额            2497 non-null float64
实采
差异         2046 non-null float64
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 180.5+ KB
None


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  0%|                                                                                         | 0/2886 [00:00<?, ?it/s]E:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

In [2]:
# 日实际采购数量/金额
# from __future__ import unicode_literals
from pyecharts import Grid
from pyecharts import Bar, Line, Overlap


data_sum = df.pivot_table(index=["采购\n日期"],values=["实际\n采购","金额"],aggfunc='sum')
data_sum.reset_index(inplace=True)
# 保留两位小数
decimals = pd.Series([1,0], index=['金额','实际\n采购'])
data_sum = data_sum.round(decimals)

attr_sum = data_sum['采购\n日期'].astype(str)#折线图的类别项不可以用datatime数据类型
v6 = data_sum['实际\n采购']
v7 = data_sum['金额']
bar_sum = Bar(title="采购数量",title_text_size=15,title_pos="6%",title_top="3%")
bar_sum.add("采购数量", attr_sum, v6,
            xaxis_label_textsize = 8,xaxis_rotate = 90,
            mark_point=[ "max", "min"],mark_point_textcolor="#000" ,mark_point_symbolsize="40",mark_point_textsize=3,
            mark_line=[ "average"],
            is_splitline_show=False,is_legend_show=False)

line_jine = Line("采购金额",title_text_size=15,title_pos="55%",title_top="3%")
line_jine.add("采购金额", attr_sum, v7,
              xaxis_label_textsize = 8,xaxis_rotate = 90,
              is_lable_show=True,is_label_emphasis=True,
              label_text_color ="#000",is_splitline_show=False,is_legend_show=False,
              mark_point=[ "max", "min"],mark_point_textcolor="red" ,mark_point_symbolsize="40",mark_point_textsize=3,
              mark_line=[ "average"],
              )

In [3]:
# 类别单价数据处理
data_catgory_detail=df.pivot_table('单价',index='采购\n日期',columns='类别',aggfunc='mean')
data_catgory_detail.reset_index(inplace=True)
data_catgory_detail
# 保留两位小数
decimals = pd.Series([1,1,1,1,1], index=['蔬菜类','猪肉类','干货类','冻货类','鸡肉类'])
data_catgory_detail = data_catgory_detail.round(decimals)

# 提取五大类单价数据
attr = data_catgory_detail['采购\n日期'].astype(str)

v1 = data_catgory_detail['猪肉类']
v2 = data_catgory_detail['鸡肉类']
v3 = data_catgory_detail['蔬菜类']
v4 = data_catgory_detail['干货类']
v5 = data_catgory_detail['冻货类']

# 每日综合采购单价图表绘制
bar = Bar("每日综合采购单价",title_text_size=15,title_pos="27%",title_top="26%")

bar.add("猪肉类", attr, v1, 
        is_splitline_show=False,axis_interval=0,xaxis_label_textsize = 8,xaxis_rotate = 90,
        legend_orient="vertical",legend_pos="68%",legend_top="28%")
bar.add("鸡肉类", attr, v2, 
        is_splitline_show=False,axis_interval=0,xaxis_label_textsize = 8,xaxis_rotate = 90,
        legend_orient="vertical",legend_pos="68%",legend_top="28%")
bar.add("蔬菜类", attr, v3, 
        axis_interval=0,xaxis_label_textsize =8,xaxis_rotate = 90,is_splitline_show=False,
        legend_pos="68%",legend_top="28%",)
bar.add("干货类", attr, v4, 
        is_splitline_show=False,axis_interval=0,xaxis_label_textsize = 8,xaxis_rotate = 90,
        legend_orient="vertical",legend_pos="68%",legend_top="28%")
bar.add("冻货类", attr, v5,
        is_splitline_show=False,axis_interval=0,xaxis_label_textsize = 8,xaxis_rotate = 90,
        legend_orient="vertical",legend_pos="68%",legend_top="28%")

In [4]:
#猪肉类每日单价
line_zhurou = Line("猪肉类",title_top="46%",title_pos="27.5%",title_text_size=15)
line_zhurou.add("",attr,v1,
                axis_interval =0,xaxis_label_textsize = 8,xaxis_rotate = 45,mark_point_symbolsize="40",
                mark_point=[ "max", "min"],mark_point_textcolor="white" ,
                mark_line=[ "average"],
                is_splitline_show=False
               )
#鸡肉类每日单价图
line_jirou = Line("鸡肉类",title_top="46%",title_pos="53%",title_text_size=15)
line_jirou.add("",attr,v2,
                axis_interval =0,xaxis_label_textsize = 8,xaxis_rotate = 45,mark_point_symbolsize="40",
                mark_point=[ "max", "min"],mark_point_textcolor="white" ,
                mark_line=[ "average"],
                is_splitline_show=False
               )
# 蔬菜类每日单价图
line_shucai = Line("蔬菜类",title_top="73%",title_pos="2.5%",title_text_size=15)
line_shucai.add("",attr,v3,
                axis_interval =0,xaxis_label_textsize = 8,xaxis_rotate = 45,
                mark_point=[ "max", "min"],mark_point_textcolor="white" ,mark_point_symbolsize="40",mark_point_textsize=3,
                mark_line=[ "average"],
                is_splitline_show=False
               )
#干货类每日单价图
line_ganhuo = Line("干货类",title_top="73%",title_pos='27.5%',title_text_size=15)
line_ganhuo.add("",attr,v4,
                axis_interval =0,xaxis_label_textsize = 8,xaxis_rotate = 45,mark_point_symbolsize="40",
                is_label_show=True,label_text_color="green",label_pos='right',
                mark_line=[ "average"],
                is_splitline_show=False
               )
# 冻货类每日单价图
line_donghuo = Line("冻货类",title_top="73%",title_pos="53%" ,title_text_size=15)
line_donghuo.add("",attr,v5,
                axis_interval =0,xaxis_label_textsize = 8,xaxis_rotate = 45,mark_point_symbolsize="40",
                is_label_show=True,label_text_color="red",label_pos='right',
                mark_line=[ "average"],
                is_splitline_show=False
               )

In [5]:
# 每日采购差异图表
data_chayi = df.pivot_table(index=["采购\n日期"],values=["实采\n差异"],aggfunc='sum')#[]添加与否与很大差异
data_chayi.reset_index(inplace=True)
line_chayi = Line("实采差异")
attr_chayi = data_chayi["采购\n日期"].astype("str") #折线图的X轴不能为时间datetime类型
v8 = data_chayi['实采\n差异']
line_chayi = Line("实采差异",title_text_size=15,title_top = "46%",title_pos="2.5%")
line_chayi.add(
    "实采差异",attr_chayi,v8,is_fill=True,line_opacity=0.2,area_opacity=0.4,symbol=None,
    xaxis_label_textsize = 8,xaxis_rotate = 45,is_splitline_show=False,is_legend_show=False)

In [10]:
### 近前一周5大类的采购占比图
data_sum_jine = df.pivot_table(index=["采购\n日期"],values="金额",columns='类别',aggfunc='sum',fill_value=0)
# data_sum_jine.reset_index(inplace=True)# 为了方便提取数据，采购日期仍为索引列
data_sum_jine.head()
# 获取距今为止前7日的数据，但由于做此图时的数据源为1月份的，为了方便取数据，只能-40+的天数
# 由于今后是每周五更新，故可以直接取最后五日的数据，而不用这样重复迭代
now_time =datetime.datetime.now()#获取当前时间
yes_time5 = (now_time -43*Day()).strftime('%Y-%m-%d')
yes_time4 = (now_time -44*Day()).strftime('%Y-%m-%d')
yes_time3 = (now_time -45*Day()).strftime('%Y-%m-%d')
yes_time2 = (now_time -46*Day()).strftime('%Y-%m-%d')
yes_time1 = (now_time -47*Day()).strftime('%Y-%m-%d')
yes_time1
# 取5日5大类的每天的订购金额
leibie = ['蔬菜类', '猪肉类', '鸡肉类','干货类', '冻货类' ]
attr_time =['蔬菜', '猪肉', '鸡肉','干货', '冻货' ] 
v1_time = data_sum_jine.ix[yes_time1,leibie].tolist()#注意ix的用法，区别于iloc,loc的用法
v2_time = data_sum_jine.ix[yes_time2,leibie].tolist()
v3_time = data_sum_jine.ix[yes_time3,leibie].tolist()
v4_time = data_sum_jine.ix[yes_time4,leibie].tolist()
v5_time = data_sum_jine.ix[yes_time5,leibie].tolist()

# 近一周5大类占比图绘制
from pyecharts import Pie,Style
pie = Pie('上周5大类采购金额占比', title_text_size=15,title_top="26%" ,title_pos='75%',height=600)
# 设置同一种风格
style = Style()
pie_style = style.add(label_pos="inside",is_label_show=True,label_text_size=8,
                      label_text_color=None,legend_orient="vertical",legend_top = "56%",legend_pos="91%")
pie.add("星期一", attr_time, v1_time, center=[85, 36], radius=[6, 13], **pie_style)
pie.add("星期二", attr_time, v2_time, center=[85, 50], radius=[6, 13], **pie_style)
pie.add("星期三", attr_time, v3_time, center=[85, 64], radius=[6, 13], **pie_style)
pie.add("星期四", attr_time, v4_time, center=[85, 78], radius=[6, 13], **pie_style)
pie.add("星期五", attr_time, v5_time, center=[85, 92], radius=[6, 13], **pie_style)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for posi

In [1]:
#合并显示
from pyecharts import Grid

grid = Grid(height=900,width=1050)

# 第一行
grid.add(bar_sum, grid_top="5%",grid_bottom="80%",grid_right="55%",grid_left="6%")
grid.add(line_jine, grid_top="5%",grid_bottom="80%",grid_right="5%",grid_left="55%")
# 第二行
grid.add(bar, grid_top="28%",grid_bottom="60%",grid_right="33%",grid_left="5%")
# 第三行
grid.add(line_chayi, grid_top="49%",grid_bottom="32%",grid_right="77%",grid_left="5%")
grid.add(line_zhurou, grid_top="49%",grid_bottom="32%",grid_right="51%",grid_left="30%",)
grid.add(line_jirou, grid_top="49%",grid_bottom="32%",grid_right="27%",grid_left="56%")
# 第四行
grid.add(line_shucai, grid_top="76%",grid_bottom="5%",grid_right="78%",grid_left="5%")
grid.add(line_ganhuo, grid_top="76%",grid_bottom="5%",grid_right="51%",grid_left="30%",)
grid.add(line_donghuo, grid_top="76%",grid_bottom="5%",grid_right="27%",grid_left="56%",)

grid.add(pie,grid_top = "28%",grid_bottom = "5%",grid_left = "82%",grid_right="1%")

grid.render()

NameError: name 'bar_sum' is not defined